In [0]:
import nltk
import numpy as np
import random
import string 

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import io

print("ASSISTENTE: Serei seu Assistente. Se você desejar sair diga tchau!")
  
#Escolha do assunto
assunto = input('Escolha o assunto da notícia desejada: ') 
if assunto == 'ambiente':
   f=io.open('/content/drive/My Drive/Colab Notebooks/chatbot-assuntos/ambiente.txt','r', encoding="utf-8")
elif assunto == 'cotidiano':
   f=io.open('/content/drive/My Drive/Colab Notebooks/chatbot-assuntos/cotidiano.txt','r', encoding="utf-8")
elif assunto == 'esporte':
   f=io.open('/content/drive/My Drive/Colab Notebooks/chatbot-assuntos/esporte.txt','r', encoding='utf8')
elif assunto == 'ilustrada':
   f=io.open('/content/drive/My Drive/Colab Notebooks/chatbot-assuntos/ilustrada.txt','r', encoding="utf-8")
elif assunto == 'mercado':
   f=io.open('/content/drive/My Drive/Colab Notebooks/chatbot-assuntos/mercado.txt','r',  encoding="utf-8")
elif assunto == 'mundo':
   f=io.open('/content/drive/My Drive/Colab Notebooks/chatbot-assuntos/mundo.txt','r', encoding='utf-8')
elif assunto == 'poder':
   f=io.open('/content/drive/My Drive/Colab Notebooks/chatbot-assuntos/poder.txt','r', encoding='utf-8')
elif assunto == 'tec':
   f=io.open('/content/drive/My Drive/Colab Notebooks/chatbot-assuntos/tec.txt','r', encoding='utf-8')
else:
   print("Ainda não tenho informações sobre esse assunto")

raw=f.read()
raw=raw.lower() # converts to lowercase
nltk.download('punkt')   
nltk.download('wordnet')
sent_tokens = nltk.sent_tokenize(raw) # converts to list of sentences 
word_tokens = nltk.word_tokenize(raw) # converts to list of words
  
#Keyword matching
SAUDACAO_ENTRADA = ("ola", "oi", "bom dia", "boa tarde", "boa noite", "ei")
RESPOSTA_SAUDACAO = ["Oi", "Olá", "Em que posso lhe ajudar?", "Estou feliz por você estar falando comigo!"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in SAUDACAO_ENTRADA:
           return random.choice(RESPOSTA_SAUDACAO)
  
#Generating Response
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize)
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"Desculpe! Ainda não sei sobre esse assunto."
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

flag=True

#Pre-processing the raw text
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='tchau'):
        if(user_response=='Obrigado' or user_response=='Espero ter ajudado'):
            flag=False
            print("ASSISTENTE: Seja bem vindo!") 
        else:
            if(greeting(user_response)!=None):
                print("ASSISTENTE: "+greeting(user_response))
            else:
                print("ASSISTENTE: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ASSISTENTE: Até logo!")



ASSISTENTE: Serei seu Assistente. Se você desejar sair diga tchau!
Escolha o assunto da notícia desejada: ilustrada
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
star wars
ASSISTENTE: filme 'star wars: os últimos jedi' ganha trailer definitivo; assista,"a disney divulgou na noite desta segunda-feira (9) o novo trailer de ""star wars: os últimos jedi"", oitavo episódio da saga.
filme
ASSISTENTE: com direção de rian johnson, o filme será lançado em 14 de dezembro no brasil.
atriz
ASSISTENTE: Desculpe! Ainda não sei sobre esse assunto.
tchau
ASSISTENTE: Até logo!
